# RFM Analysis

## Model Process
- [Connect to SQL](#ConnectSQL)


<a id='ConnectSQL'></a>
### Connect to SQL 

In [ ]:
#import SQL library
import pyodbc
import pandas as pd #and dont forget pandas!

In [2]:
#establishes an open connection to SQL
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-1E730VKQ\MSSQLSERVER03;'
                      'Database=AdventureWorks2017;'
                      'Trusted_Connection=yes;')

In [3]:
#pull customer and product purchases from SQL including region and demographics for clustering recommendations

pythonSQLobject=pd.read_sql_query("""
SELECT *
FROM [Sales].[SalesOrderHeader]
""", conn)



conn.close() # please close it after !

# Data Exploration

In [4]:
pythonSQLobject.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43659,PO522145787,10-4020-000676,...,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,None,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07
1,43660,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,...,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
2,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,...,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
3,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,...,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
4,43665,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43665,PO16588191572,10-4020-000146,...,15232.0,35568Vi78804,NaN,14352.7713,1375.9427,429.9821,16158.6961,None,5602C304-853C-43D7-9E79-76E320D476CF,2011-06-07


In [5]:
pythonSQLobject.shape

(31462, 26)

In [6]:
df = pythonSQLobject

In [7]:
df.columns

Index(['SalesOrderID', 'RevisionNumber', 'OrderDate', 'DueDate', 'ShipDate',
       'Status', 'OnlineOrderFlag', 'SalesOrderNumber', 'PurchaseOrderNumber',
       'AccountNumber', 'CustomerID', 'SalesPersonID', 'TerritoryID',
       'BillToAddressID', 'ShipToAddressID', 'ShipMethodID', 'CreditCardID',
       'CreditCardApprovalCode', 'CurrencyRateID', 'SubTotal', 'TaxAmt',
       'Freight', 'TotalDue', 'Comment', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [15]:
SalesOrders=df.sort_values(['CustomerID','OrderDate'])

In [17]:
SalesOrders.CustomerID.head()

131      11000
7860     11000
13756    11000
105      11001
7831     11001
Name: CustomerID, dtype: int64

# Transactional Summary

In [56]:
#get transactional summary - we will need to model these additional features later as possible predictors for churn
SalesSummary = SalesOrders.groupby('CustomerID').agg({'TotalDue':['count','sum'],'OrderDate':'max'})

In [57]:
SalesSummary

TotalDue               OrderDate
              count          sum        max
CustomerID                                 
11000             3    9115.1341 2013-10-03
11001             3    7054.1875 2014-05-12
11002             3    8966.0143 2013-07-26
11003             3    8993.9155 2013-10-10
11004             3    9056.5911 2013-10-01
11005             3    8974.0698 2013-10-02
11006             3    8971.5283 2013-10-14
11007             3    9073.1551 2013-08-19
11008             3    8957.4726 2013-08-02
11009             3    8940.9197 2013-10-09
11010             3    8937.2843 2013-10-23
11011             3    8987.0093 2013-08-19
11012             2      89.7923 2014-03-17
11013             2     125.9258 2014-06-23
11014             2     152.9873 2013-09-30
11015             1    2763.5719 2013-06-20
11016             1    2577.1694 2013-07-12
11017             3    7109.9127 2014-03-16
11018             3    7219.2745 2014-03-26
11019            17     975.3840 2014-06-14
11020             1    2560.2519 2013-05-31
11021             1    2621.0158 2013-06-25
11022             1    2566.1194 2013-06-22
11023             2     135.0752 2014-06-16
11024             2      62.4436 2013-12-26
11025             3    7267.3641 2014-03-27
11026             3    7266.2481 2014-03-17
11027             3    7283.6849 2014-03-26
11028             3    7154.4552 2014-03-09
11029             3    7254.6456 2014-04-13
...             ...          ...        ...
30089             4    5894.0226 2014-05-01
30090             4   93981.0186 2012-04-30
30091             8   99385.9745 2014-03-31
30092             4  130063.1207 2014-05-01
30093             4  111168.3274 2014-03-31
30094             8  381763.5549 2014-03-31
30095            12  257252.7645 2014-03-31
30096             9  148385.9496 2014-03-01
30097             4  120560.4446 2014-05-01
30098             2      65.5890 2012-11-30
30099             1    1459.2824 2013-04-30
30100             4   99440.3238 2012-03-30
30101             4  166847.2483 2014-03-31
30102             4    7095.0672 2014-05-01
30103             8  725867.1659 2014-03-31
30104             4    6322.6323 2013-02-28
30105             8  195196.9483 2014-05-01
30106             8  113981.9967 2013-04-30
30107            12  650362.0535 2014-03-31
30108             4  162979.1262 2013-03-30
30109             8  378936.0887 2014-03-01
30110             4    1833.5947 2013-04-30
30111             4  311393.1457 2012-03-30
30112             8  538333.4655 2014-03-31
30113             8  307946.0796 2014-05-01
30114             8   13135.0333 2014-03-01
30115             8   10033.8172 2014-03-31
30116             4  211671.2674 2014-03-01
30117            12  919801.8188 2014-03-31
30118             8  313671.5352 2014-05-01

[19119 rows x 3 columns]

In [58]:
SalesSummary.columns

MultiIndex(levels=[['TotalDue', 'OrderDate'], ['count', 'max', 'sum']],
           codes=[[0, 0, 1], [0, 2, 1]])

In [59]:
#flatten column headers
SalesSummary.columns = ["_".join(SalesSummary) for SalesSummary in SalesSummary.columns.ravel()]

In [60]:
SalesSummary.columns

Index(['TotalDue_count', 'TotalDue_sum', 'OrderDate_max'], dtype='object')

# Get most recent purchase date for each customer

In [61]:
LastTransaction = max(SalesOrders.OrderDate)

In [62]:
LastTransaction

Timestamp('2014-06-30 00:00:00')

In [63]:
#alternative approach

from datetime import date
myDate = date.today()

In [64]:
myDate

datetime.date(2019, 9, 17)

In [65]:
#here we apply date difference between each customer's last order and last transcation in DB
#SalesSummary['OrderDate_max'] = pd.to_datetime(SalesSummary['OrderDate_max'], format='%Y-%m-%d')

def DaysElapsed(LastOrder):
    return LastTransaction - LastOrder

SalesSummary['DaysSinceLastOrder'] = SalesSummary['OrderDate_max'].apply(DaysElapsed)

In [66]:
SalesSummary.head()

,TotalDue_count,TotalDue_sum,OrderDate_max,DaysSinceLastOrder
CustomerID,,,,
11000,3,9115.1341,2013-10-03,270 days
11001,3,7054.1875,2014-05-12,49 days
11002,3,8966.0143,2013-07-26,339 days
11003,3,8993.9155,2013-10-10,263 days
11004,3,9056.5911,2013-10-01,272 days


In [67]:
SalesSummary.drop('OrderDate_max',axis=1,inplace=True)

In [68]:
SalesSummary.head()

,TotalDue_count,TotalDue_sum,DaysSinceLastOrder
CustomerID,,,
11000,3,9115.1341,270 days
11001,3,7054.1875,49 days
11002,3,8966.0143,339 days
11003,3,8993.9155,263 days
11004,3,9056.5911,272 days


In [69]:
SalesSummary.head()

,TotalDue_count,TotalDue_sum,DaysSinceLastOrder
CustomerID,,,
11000,3,9115.1341,270 days
11001,3,7054.1875,49 days
11002,3,8966.0143,339 days
11003,3,8993.9155,263 days
11004,3,9056.5911,272 days


In [70]:
#RFM analysis - rename columns

SalesSummary.rename(columns={'TotalDue_count':'Frequency',
                          'TotalDue_sum':'Monetary',
                          'DaysSinceLastOrder':'Recency'}, 
                 inplace=True)

In [71]:
SalesSummary.head()

,Frequency,Monetary,Recency
CustomerID,,,
11000,3,9115.1341,270 days
11001,3,7054.1875,49 days
11002,3,8966.0143,339 days
11003,3,8993.9155,263 days
11004,3,9056.5911,272 days
